In [ ]:
! pip install wandb
# grab @TheodoreGalanos's fork for config file
! git clone https://github.com/TheodoreGalanos/gpt-neox/
import wandb
import pandas as pd
import numpy as np

In [ ]:
# initialize config
sweep_config = {
  "name": "Scaling laws sweep",
  "method": "grid",
  "parameters": {
    "valid_set": {
        "distribution": "categorical",
        'values': [
          ### This will be a list of strings
          ### with the following values:
          ### N, num_layers, num_attention_heads, hidden_size, lr
          ### separated by commas
      ]
    },
  }
}

# read TG's config
df = pd.read_csv(
    './gpt-neox/configs/scaling_experiment/config_parameters.csv',
    )

# create strings for each row
for i in range(len(df)):
  row = df.iloc[i]
  row['N'] = 12*row['n_layer']*row['n_embd']**2
  # add LR according equation D.1 from Kaplan et. al
  # "Scaling Laws for Neural Language Models"
  row['lr'] = 0.003239 + (-0.0001395)*np.log(row['N'])
  row.drop(['Exponent', 'head_state'], inplace=True)
  row = row[['N', 'n_layer', 'n_head', 'n_embd', 'lr']]
  # concatenate into a string to be split later
  row = ','.join([str(x) for x in row])
  sweep_config['parameters']['valid_set']['values'].append(row)

In [6]:
# check source
print(df)

# and config
sweep_config

    Exponent  n_layer  n_head  n_embd  head_state
0         11        1       1      72          72
1         11        2       1      48          48
2         11        2       2      48          24
3         11        2       2      48          24
4         11        2       2      48          24
5         12        1       2     112          56
6         12        2       2      88          44
7         12        3       2      64          32
8         13        2       4     128          32
9         13        2       2     128          64
10        13        2       4     128          32
11        14        2       4     224          56
12        14        2       7     224          32
13        14        6       4     128          32
14        14        2      16     256          16
15        15        1       8     512          64
16        15        4       8     256          32
17        15        4       4     256          64
18        15        3       8     256          32


{'method': 'grid',
 'name': 'Scaling laws sweep',
 'parameters': {'valid_set': {'distribution': 'categorical',
   'values': ['62208.0,1.0,1.0,72.0,0.0016991656751490947',
    '55296.0,2.0,1.0,48.0,0.0017155964086231603',
    '55296.0,2.0,2.0,48.0,0.0017155964086231603',
    '55296.0,2.0,2.0,48.0,0.0017155964086231603',
    '55296.0,2.0,2.0,48.0,0.0017155964086231603',
    '150528.0,1.0,2.0,112.0,0.0015758943372632425',
    '185856.0,2.0,2.0,88.0,0.001546484519427042',
    '147456.0,3.0,2.0,64.0,0.0015787707278280244',
    '393216.0,2.0,4.0,128.0,0.0014419450470328887',
    '393216.0,2.0,2.0,128.0,0.0014419450470328887',
    '393216.0,2.0,4.0,128.0,0.0014419450470328887',
    '1204224.0,2.0,4.0,224.0,0.0012858122421989054',
    '1204224.0,2.0,7.0,224.0,0.0012858122421989054',
    '1179648.0,6.0,4.0,128.0,0.0012886886327636872',
    '1572864.0,2.0,16.0,256.0,0.001248556983656664',
    '3145728.0,1.0,8.0,512.0,0.0011518629519685513',
    '3145728.0,4.0,8.0,256.0,0.0011518629519685513',
  

In [5]:
# test run
sweep_id = wandb.sweep(sweep_config)

def train():
    run = wandb.init()
    print(run.config.valid_set)
    vars = {k:v for k,v in zip(
        # these are from neox_arguments.md
        ['N',
         'num_layers',
         'num_attention_heads',
         'hidden_size',
         'head_state',
         'lr'
         ],
        [float(x) for x in run.config.valid_set.split(',')]
    )}
    print([x for x in zip(vars.keys(),vars.values())])
    run.finish()

sweep_id = wandb.sweep(sweep_config)
agent = wandb.agent(sweep_id=sweep_id, function=train)
agent.run()

Create sweep with ID: bufkhsip
Sweep URL: https://wandb.ai/dontclickmebro/uncategorized/sweeps/bufkhsip
Create sweep with ID: uw2abv80
Sweep URL: https://wandb.ai/dontclickmebro/uncategorized/sweeps/uw2abv80


wandb: Agent Starting Run: z5d83uwi with config:
wandb: 	valid_set: 62208.0,1.0,1.0,72.0,0.0016991656751490947
/usr/lib/python3.7/multiprocessing/semaphore_tracker.py:55: UserWarning: semaphore_tracker: process died unexpectedly, relaunching.  Some semaphores might leak.
  warnings.warn('semaphore_tracker: process died unexpectedly, '


62208.0,1.0,1.0,72.0,0.0016991656751490947
[('N', 62208.0), ('num_layers', 1.0), ('num_attention_heads', 1.0), ('hidden_size', 72.0), ('head_state', 0.0016991656751490947)]


wandb: Agent Starting Run: 0gzalme6 with config:
wandb: 	valid_set: 55296.0,2.0,1.0,48.0,0.0017155964086231603


wandb: Ctrl + C detected. Stopping sweep.
Unhandled exception in thread started by <bound method Thread._bootstrap of <Thread(Thread-28, initial daemon)>>


AttributeError: ignored